In [1]:
import os
import xarray as xr
import numpy as np
import healpy as hp
from easygems import healpix as egh
from datetime import datetime


In [2]:
import os
import xarray as xr

# Define zoom
zoom = 10

# Define required variables
variables = ['pr', 'uas', 'vas', 'ps', 'tas', 'orog']

# Define ICON & UM Models paths
GE_path = "/g/data/qx55/germany_node/d3hp003.zarr"
GE_file = f"P1D_mean_z{zoom}_atm.zarr"

UK_path = "/g/data/qx55/uk_node/glm.n2560_RAL3p3/"
UK_file = f"data.healpix.PT1H.z{zoom}.zarr"

# Load files
ge = xr.open_zarr(os.path.join(GE_path, GE_file))
uk = xr.open_zarr(os.path.join(UK_path, UK_file))

# Optional: filter only available variables
ge_vars = [v for v in variables if v in ge.variables]
uk_vars = [v for v in variables if v in uk.variables]
ge = ge[ge_vars]
uk = uk[uk_vars]

# Crop to required time range
start_time = "2021-02-01"
end_time = "2021-02-10"
ge = ge.sel(time=slice(start_time, end_time))
uk = uk.sel(time=slice(start_time, end_time))

# Keep original for attribute copying later
ge_orig = ge.copy(deep=True)
uk_orig = uk.copy(deep=True)

# Remove all attributes
for ds in [ge, uk]:
    ds.attrs = {}  # Remove global attributes
    for var in ds.data_vars:
        ds[var].attrs = {}  # Remove all variable-level attributes

# Function to selectively copy attributes
def copy_attributes(original_ds, new_ds):
    """Copy only 'unit', 'units', and 'long_name' attributes from original_ds to new_ds"""
    for var in original_ds.data_vars:
        if var in new_ds:
            for attr in ['units', 'unit', 'long_name']:
                if attr in original_ds[var].attrs:
                    new_ds[var].attrs[attr] = original_ds[var].attrs[attr]
    return new_ds

# Restore selected attributes
ge = copy_attributes(ge_orig, ge)
uk = copy_attributes(uk_orig, uk)


In [3]:
ge

<xarray.Dataset> Size: 3GB
Dimensions:  (time: 10, cell: 12582912)
Coordinates:
  * time     (time) datetime64[ns] 80B 2021-02-01 2021-02-02 ... 2021-02-10
Dimensions without coordinates: cell
Data variables:
    pr       (time, cell) float32 503MB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    uas      (time, cell) float32 503MB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    vas      (time, cell) float32 503MB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    ps       (time, cell) float32 503MB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    tas      (time, cell) float32 503MB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    orog     (cell) float32 50MB dask.array<chunksize=(12582912,), meta=np.ndarray>

In [4]:
uk

<xarray.Dataset> Size: 61GB
Dimensions:  (time: 240, cell: 12582912)
Coordinates:
  * cell     (cell) int64 101MB 0 1 2 3 ... 12582908 12582909 12582910 12582911
    crs      float64 8B ...
  * time     (time) datetime64[ns] 2kB 2021-02-01 ... 2021-02-10T23:00:00
Data variables:
    pr       (time, cell) float32 12GB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    uas      (time, cell) float32 12GB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    vas      (time, cell) float32 12GB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    ps       (time, cell) float32 12GB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    tas      (time, cell) float32 12GB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    orog     (cell) float64 101MB dask.array<chunksize=(98304,), meta=np.ndarray>

In [5]:
# Create orography datasets
uk_orog = uk.orog
ge_orog = ge.orog

uk = uk.drop_vars('orog')
ge = ge.drop_vars('orog')


In [6]:
# Print units for each variable in the dataset
ds = ge
for var in ds.data_vars:
    if 'units' in ds[var].attrs:
        print(f"Variable: {var}, Units: {ds[var].attrs['units']}")
    else:
        print(f"Variable: {var} does not have units attribute.")


Variable: pr, Units: kg m-2 s-1
Variable: uas, Units: m s-1
Variable: vas, Units: m s-1
Variable: ps, Units: Pa
Variable: tas, Units: K


In [7]:
# Define function to compute time averages
def seasonal_means(ds):
    ds['month'] = ds['time'].dt.month
    return {
        'DJF': ds.sel(time=ds['month'].isin([12, 1, 2])).mean(dim='time'),
        'MAM': ds.sel(time=ds['month'].isin([3, 4, 5])).mean(dim='time'),
        'JJA': ds.sel(time=ds['month'].isin([6, 7, 8])).mean(dim='time'),
        'SON': ds.sel(time=ds['month'].isin([9, 10, 11])).mean(dim='time'),
    }

seasons_uk = seasonal_means(uk)
seasons_ge = seasonal_means(ge)


/scratch/nf33/public/miniconda/envs/digital_earths_env/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/scratch/nf33/public/miniconda/envs/digital_earths_env/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/scratch/nf33/public/miniconda/envs/digital_earths_env/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/scratch/nf33/public/miniconda/envs/digital_earths_env/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/scratch/nf33/public/miniconda/envs/digital_earths_env/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar

In [8]:
def regrid_function(ds):
    lon = np.linspace(125, 235, num=int((235 - 125)/0.04) + 1)
    lat = np.linspace(-35, 25, num=int((25 - (-35))/0.04) + 1)

    # Determine nside from 'cell'-dimmed variable
    if isinstance(ds, xr.Dataset):
        for var in ds.data_vars:
            if 'cell' in ds[var].dims:
                nside = hp.get_nside(ds[var].values)
                break
        else:
            raise ValueError("No variable with 'cell' dimension found to infer nside.")
    elif isinstance(ds, xr.DataArray):
        if 'cell' in ds.dims:
            nside = hp.get_nside(ds.values)
        else:
            raise ValueError("DataArray must have 'cell' dimension.")
    else:
        raise TypeError("Input must be an xarray Dataset or DataArray.")

    # Build nearest neighbor index array
    def get_nn_lon_lat_index(nside, lons, lats):
        lons2, lats2 = np.meshgrid(lons, lats)
        return xr.DataArray(
            hp.ang2pix(nside, lons2, lats2, nest=True, lonlat=True),
            coords=[("lat", lats), ("lon", lons)],
        )

    cells = get_nn_lon_lat_index(nside, lon, lat)

    # Apply regridding
    return ds.isel(cell=cells)

# Regrid Climate Datasets
uk_regridded = {season: regrid_function(ds) for season, ds in seasons_uk.items()}
ge_regridded = {season: regrid_function(ds) for season, ds in seasons_ge.items()}

# Regrid Oroggraphy Datasets
uk_orog_regridded = regrid_function(uk_orog)
ge_orog_regridded = regrid_function(ge_orog)

In [9]:
def clean_all_attrs(ds):
    """Remove or sanitize illegal NetCDF attributes from dataset and its variables"""
    def sanitize(attrs):
        clean = {}
        for k, v in attrs.items():
            try:
                if isinstance(v, (np.generic, np.bool_)):
                    v = v.item()
                if isinstance(v, (str, int, float)):
                    clean[k] = v
            except Exception:
                continue
        return clean

    ds.attrs = sanitize(ds.attrs)
    for var in ds.data_vars:
        ds[var].attrs = sanitize(ds[var].attrs)
    return ds


In [10]:
def copy_attributes(original_ds, new_ds):
    """Copy only 'unit' and 'long_name' attributes from original_ds to new_ds"""
    for var in original_ds.data_vars:
        if var in new_ds:
            for attr in ['units', 'unit', 'long_name']:
                if attr in original_ds[var].attrs:
                    new_ds[var].attrs[attr] = original_ds[var].attrs[attr]
    return new_ds


In [11]:
# Output path
record_path = '/scratch/nf33/hk25-ConvZones'

def save_topography(model_name, orog_da):
    # Wrap DataArray in a Dataset if needed
    if isinstance(orog_da, xr.DataArray):
        orog_ds = orog_da.to_dataset(name='orog')
    else:
        orog_ds = orog_da

    # Clean attributes before saving
    orog_ds = clean_all_attrs(orog_ds)

    # Define filename and path
    filename = f"{model_name}_topo_Z10_test.nc"
    filepath = os.path.join(record_path, filename)

    # Compression settings
    encoding = {'orog': {'zlib': True, 'complevel': 2}}

    # Write NetCDF
    orog_ds.to_netcdf(filepath, encoding=encoding)
    print(f"Saved: {filepath}")

save_topography("UM", uk_orog_regridded)
save_topography("ICON", ge_orog_regridded)


Saved: /scratch/nf33/hk25-ConvZones/UM_topo_Z10_test.nc
Saved: /scratch/nf33/hk25-ConvZones/ICON_topo_Z10_test.nc


In [12]:
# Output path
record_path = '/scratch/nf33/hk25-ConvZones'

# Save function
def save_seasonal_regridded(model_name, original_ds, regridded_dict):
    for season, ds in regridded_dict.items():
        # Copy variable attributes
        ds_with_attrs = copy_attributes(original_ds, ds)

        # Define filename
        season_tag = 'tot' if season == 'full' else season.lower()
        filename = f"{model_name}_{season_tag}_avg_202003_202102_Z10_test.nc"
        filepath = os.path.join(record_path, filename)

        # Compression settings
        encoding = {var: {'zlib': True, 'complevel': 2} for var in ds_with_attrs.data_vars}

        # Write NetCDF
        ds_with_attrs.to_netcdf(filepath, encoding=encoding)
        print(f"Saved: {filepath}")

# Save GE (ICON) regridded data
save_seasonal_regridded("ICON", ge, ge_regridded)

# Save UK (UM) regridded data
save_seasonal_regridded("UM", uk, uk_regridded)


Saved: /scratch/nf33/hk25-ConvZones/ICON_djf_avg_202003_202102_Z10_test.nc
Saved: /scratch/nf33/hk25-ConvZones/ICON_mam_avg_202003_202102_Z10_test.nc
Saved: /scratch/nf33/hk25-ConvZones/ICON_jja_avg_202003_202102_Z10_test.nc
Saved: /scratch/nf33/hk25-ConvZones/ICON_son_avg_202003_202102_Z10_test.nc
Saved: /scratch/nf33/hk25-ConvZones/UM_djf_avg_202003_202102_Z10_test.nc
Saved: /scratch/nf33/hk25-ConvZones/UM_mam_avg_202003_202102_Z10_test.nc
Saved: /scratch/nf33/hk25-ConvZones/UM_jja_avg_202003_202102_Z10_test.nc
Saved: /scratch/nf33/hk25-ConvZones/UM_son_avg_202003_202102_Z10_test.nc


In [13]:
import xarray as xr
import glob
import os

# Define the base path and file patterns
record_path = '/scratch/nf33/hk25-ConvZones'
uk_pattern = os.path.join(record_path, 'UM_*_avg_202003_202102_Z10_test.nc')
ge_pattern = os.path.join(record_path, 'ICON_*_avg_202003_202102_Z10_test.nc')

# Find all matching files
uk_files = sorted(glob.glob(uk_pattern))
ge_files = sorted(glob.glob(ge_pattern))

# Load and average UK datasets
uk_datasets = [xr.open_dataset(f) for f in uk_files]
uk_combined = xr.concat(uk_datasets, dim='ensemble')
uk_avg = uk_combined.mean(dim='ensemble')

# Load and average GE datasets
ge_datasets = [xr.open_dataset(f) for f in ge_files]
ge_combined = xr.concat(ge_datasets, dim='ensemble')
ge_avg = ge_combined.mean(dim='ensemble')

# Compression settings
comp = dict(zlib=True, complevel=2)
encoding = {var: comp for var in uk_avg.data_vars}

# Save averaged datasets
uk_avg.to_netcdf(os.path.join(record_path, 'UM_tot_avg_202003_202102_Z10_test.nc'),
                 encoding=encoding)
ge_avg.to_netcdf(os.path.join(record_path, 'ICON_tot_avg_202003_202102_Z10_test.nc'),
                 encoding=encoding)
